<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M0.532 · Pattern Recognition</p>
<p style="margin: 0; text-align:right;">Computational Engineering and Mathematics Master</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Computers, Multimedia and Telecommunications Department</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

In this notebook, we will use [PytorchVideo](https://pytorchvideo.org/), a deep learning library for video understanding research. More specifically, we will deal with the action recognition task. This task consists on recognizing which action is being done in a video.






First of all, we need to install the PytorchVideo library.

In [ ]:
pip install pytorchvideo

     |████████████████████████████████| 128 kB 7.6 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 
     |████████████████████████████████| 37.2 MB 1.2 MB/s 
     |████████████████████████████████| 596 kB 43.0 MB/s 
  Created wheel for pytorchvideo: filename=pytorchvideo-0.1.3-py3-none-any.whl size=183829 sha256=e749422ab8af6681b2f002aaca9be3f8328c95d807c3082ece93be2786993af4
  Stored in directory: /root/.cache/pip/wheels/d4/a7/4c/bada8b1065ae9befac2da6d7f6648cd6718681eb7901ca226d
  Created wheel for fvcore: filename=fvcore-0.1.5.post20211023-py3-none-any.whl size=60947 sha256=daadc516a3132015c65b888ae3f073287ded849009746c95ac4c4045f6629efa
  Stored in directory: /root/.cache/pip/wheels/16/98/fc/252d62cab6263c719120e06b28f3378af59b52ce7a20e81852
Successfully built pytorchvideo fvcore
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Then, we import some helper functions included in PytorchVideo to make some data transformations.

In [ ]:
import torch
import json
from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
    UniformCropVideo
)
from typing import Dict

We download a video that we will use to recognize which action is taking place.

In [ ]:
!wget https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4

--2021-12-18 20:22:46--  https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549197 (536K) [video/mp4]
Saving to: ‘archery.mp4’

archery.mp4         100%[===================>] 536.33K  --.-KB/s    in 0.1s    

2021-12-18 20:22:46 (4.61 MB/s) - ‘archery.mp4’ saved [549197/549197]



We specify that we will use the SlowFast pretrained model for action recognition and that we will use it for inference (eval method).

In [ ]:
# Device on which to run the model
# Set to cuda to load on GPU
device = "cpu"

# Pick a pretrained model and load the pretrained weights
model_name = "slowfast_r50"
model = torch.hub.load("facebookresearch/pytorchvideo", model=model_name, pretrained=True)

# Set to eval mode and move to desired device
model = model.to(device)
model = model.eval()

Downloading: "https://github.com/facebookresearch/pytorchvideo/archive/main.zip" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/SLOWFAST_8x8_R50.pyth" to /root/.cache/torch/hub/checkpoints/SLOWFAST_8x8_R50.pyth


  0%|          | 0.00/264M [00:00<?, ?B/s]

We download the action category names from Kinetcs Dataset, which is the dataset where the model has been pretrained.

In [ ]:
!wget https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json

--2021-12-18 20:13:47--  https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10326 (10K) [text/plain]
Saving to: ‘kinetics_classnames.json’

kinetics_classnames 100%[===================>]  10.08K  --.-KB/s    in 0s      

2021-12-18 20:13:47 (55.3 MB/s) - ‘kinetics_classnames.json’ saved [10326/10326]



Then, we create an id to label name mapping.

In [ ]:
with open("kinetics_classnames.json", "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

Next, we define a serie of transforms. Videos are first uniformly sampled to get a set of frames. Then, the frames are normalized, rescaled and center cropped. Finally, the frames are converted to a list of tensors.

In [ ]:
####################
# SlowFast transform
####################

side_size = 256
mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
crop_size = 256
num_frames = 32
sampling_rate = 2
frames_per_second = 30
alpha = 4

class PackPathway(torch.nn.Module):
    """
    Transform for converting video frames as a list of tensors.
    """
    def __init__(self):
        super().__init__()

    def forward(self, frames: torch.Tensor):
        fast_pathway = frames
        # Perform temporal sampling from the fast pathway.
        slow_pathway = torch.index_select(
            frames,
            1,
            torch.linspace(
                0, frames.shape[1] - 1, frames.shape[1] // alpha
            ).long(),
        )
        frame_list = [slow_pathway, fast_pathway]
        return frame_list

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(num_frames),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(
                size=side_size
            ),
            CenterCropVideo(crop_size),
            PackPathway()
        ]
    ),
)

# The duration of the input clip is also specific to the model.
clip_duration = (num_frames * sampling_rate)/frames_per_second

We select the video that we want to use and we apply the previous transforms to normalize it.

In [ ]:
# Load the example video
video_path = "archery.mp4"

# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0
end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class
video = EncodedVideo.from_path(video_path)

# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
video_data = transform(video_data)

# Move the inputs to the desired device
inputs = video_data["video"]
inputs = [i.to(device)[None, ...] for i in inputs]

The list of tensors returned by the previous code (inputs) is passed through the model to get the predictions (preds).

In [ ]:
# Pass the input clip through the model
preds = model(inputs)

We get the top-5 predicted action categories with the corresponding confidence score values.

In [ ]:
# Get the predicted classes
post_act = torch.nn.Softmax(dim=1)
preds_max = post_act(preds)
pred_classes = preds_max.topk(k=5).indices

# Map the predicted classes to the label names
pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes[0]]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print(preds_max.topk(k=5))

Predicted labels: archery, throwing axe, playing paintball, disc golfing, riding or walking with horse
torch.return_types.topk(
values=tensor([[1.0000e+00, 1.6928e-19, 5.2984e-22, 2.7325e-23, 2.4635e-23]],
       grad_fn=<TopkBackward0>),
indices=tensor([[  5, 356, 240,  92, 273]]))


Let's try it with another input video.

In [ ]:
!wget https://www.bogotobogo.com/python/OpenCV_Python/images/mean_shift_tracking/slow_traffic_small.mp4

In [ ]:
# Load the example video
video_path = "slow_traffic_small.mp4"

# Select the duration of the clip to load by specifying the start and end duration
# The start_sec should correspond to where the action occurs in the video
start_sec = 0
end_sec = start_sec + clip_duration

# Initialize an EncodedVideo helper class
video = EncodedVideo.from_path(video_path)

# Load the desired clip
video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)

# Apply a transform to normalize the video input
video_data = transform(video_data)

# Move the inputs to the desired device
inputs = video_data["video"]
inputs = [i.to(device)[None, ...] for i in inputs]

In [ ]:
# Pass the input clip through the model
preds = model(inputs)

In [ ]:
# Get the predicted classes
post_act = torch.nn.Softmax(dim=1)
preds_max = post_act(preds)
pred_classes = preds_max.topk(k=5).indices

# Map the predicted classes to the label names
pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes[0]]
print("Predicted labels: %s" % ", ".join(pred_class_names))
print(preds_max.topk(k=5))

Predicted labels: driving car, motorcycling, pushing car, texting, changing wheel
torch.return_types.topk(
values=tensor([[9.9150e-01, 7.3789e-03, 9.4220e-04, 1.0740e-04, 4.3958e-05]],
       grad_fn=<TopkBackward0>),
indices=tensor([[103, 199, 261, 355,  53]]))
